# Ejercicio 8: Bases de Datos Vectoriales

Las bases de datos vectoriales permiten almacenar y recuperar información representada como vectores en espacios de alta dimensión. Primero vamos a revisar los fundamentos matemáticos en los que se basan.

## 1. Espacios Vectoriales

Cada documento, imagen, o consulta se representa como un vector real en un espacio ℝ^n:

$\[ \vec{d} = [d_1, d_2, \dots, d_n] \in \mathbb{R}^n \]$

Donde $\( n \)$ suele ser 384, 768 o 1536, dependiendo del modelo de embeddings utilizado.

In [2]:
from sklearn.datasets import fetch_20newsgroups
import pandas as pd 

In [11]:
%pip install numpy==1.24.4 --only-binary :all:



   ---------------------------------------- 0.0/14.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/14.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/14.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/14.8 MB ? eta -:--:--
    --------------------------------------- 0.3/14.8 MB ? eta -:--:--
    --------------------------------------- 0.3/14.8 MB ? eta -:--:--
   - -------------------------------------- 0.5/14.8 MB 541.6 kB/s eta 0:00:27
   - -------------------------------------- 0.5/14.8 MB 541.6 kB/s eta 0:00:27
   -- ------------------------------------- 0.8/14.8 MB 621.2 kB/s eta 0:00:23
   -- ------------------------------------- 0.8/14.8 MB 621.2 kB/s eta 0:00:23
   -- ------------------------------------- 0.8/14.8 MB 621.2 kB/s eta 0:00:23
   -- ------------------------------------- 0.8/14.8 MB 621.2 kB/s eta 0:00:23
   -- ------------------------------------- 1.0/14.8 MB 474.9 kB/s eta 0:00:30
   --- --------------------

In [12]:
newsgroups_data = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))


________________________________________________________________________________
Cache loading failed
________________________________________________________________________________
No module named 'numpy._core'


KeyboardInterrupt: 

In [ ]:
# Paso 2: Crear un DataFrame solo con los documentos de texto
newgroupsdocs_df = pd.DataFrame({'raw': newsgroups_data.data})

# (Opcional) Visualiza las primeras filas
print(newgroupsdocs_df.head())

                                                 raw
0  \n\nI am sure some bashers of Pens fans are pr...
1  My brother is in the market for a high-perform...
2  \n\n\n\n\tFinally you said what you dream abou...
3  \nThink!\n\nIt's the SCSI card doing the DMA t...
4  1)    I have an old Jasmine drive which I cann...


In [ ]:
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\steve\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\steve\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [ ]:
from nltk.tokenize import word_tokenize

In [ ]:
def process_docs(docs):
    stop_words = set(stopwords.words('english'))
    words = word_tokenize(docs)
    word_filtered = [w for w in words if not w in stop_words]
    return ' '.join(word_filtered)


In [ ]:
newgroupsdocs_df['processed'] = newgroupsdocs_df['raw'].apply(process_docs)
newgroupsdocs_df

,raw,processed
0,\n\nI am sure some bashers of Pens fans are pr...,I sure bashers Pens fans pretty confused lack ...
1,My brother is in the market for a high-perform...,My brother market high-performance video card ...
2,\n\n\n\n\tFinally you said what you dream abou...,Finally said dream . Mediterranean ? ? ? ? Tha...
3,\nThink!\n\nIt's the SCSI card doing the DMA t...,Think ! It 's SCSI card DMA transfers NOT disk...
4,1) I have an old Jasmine drive which I cann...,1 ) I old Jasmine drive I use new system . My ...
...,...,...
18841,DN> From: nyeda@cnsvax.uwec.edu (David Nye)\nD...,DN > From : nyeda @ cnsvax.uwec.edu ( David Ny...
18842,\nNot in isolated ground recepticles (usually ...,Not isolated ground recepticles ( usually unus...
18843,I just installed a DX2-66 CPU in a clone mothe...,"I installed DX2-66 CPU clone motherboard , tri..."
18844,\nWouldn't this require a hyper-sphere. In 3-...,"Would n't require hyper-sphere . In 3-space , ..."


In [ ]:
import 

In [ ]:
import numpy as np

# Simulamos 3 documentos como vectores en R^3
doc1 = np.array([0.2, 0.1, 0.5])
doc2 = np.array([-0.1, 0.4, 0.3])
query = np.array([0.1, 0.3, 0.4])

print("Documentos:", doc1, doc2)
print("Consulta:", query)

Documentos: [0.2 0.1 0.5] [-0.1  0.4  0.3]
Consulta: [0.1 0.3 0.4]


## 2. Medidas de Similitud

El principio básico de una base vectorial es buscar elementos cuyo vector esté "cerca" del vector de consulta. Existen varias formas de medir esta cercanía:

### a. Distancia Euclidiana (L2)

$\[ \text{dist}(⇡\vec{q}, \vec{d}) = \sqrt{\sum_{i=1}^n (q_i - d_i)^2} \]$

Utilizada cuando los vectores no están normalizados. Implementada por defecto en `FAISS` con `IndexFlatL2`.

### b. Similitud Coseno

$\[ \cos(\theta) = \frac{\vec{q} \cdot \vec{d}}{\|\vec{q}\| \cdot \|\vec{d}\|} \]$

Esta métrica es ideal cuando se desea medir ángulos (dirección) en lugar de magnitudes. Se usa en `ChromaDB` y también puede simularse en FAISS si los vectores están normalizados.

Existe una relación entre ambas (cuando los vectores están normalizados):
$\[ \text{dist}_{\text{L2}}^2 = 2 - 2 \cdot \cos(\theta) \]$

In [ ]:
from numpy.linalg import norm

dist1 = norm(query - doc1)
dist2 = norm(query - doc2)

print("Distancia Euclidiana a doc1:", dist1)
print("Distancia Euclidiana a doc2:", dist2)

def cosine_similarity(a, b):
    return np.dot(a, b) / (norm(a) * norm(b))

sim1 = cosine_similarity(query, doc1)
sim2 = cosine_similarity(query, doc2)

print("Similitud coseno con doc1:", sim1)
print("Similitud coseno con doc2:", sim2)

Distancia Euclidiana a doc1: 0.2449489742783178
Distancia Euclidiana a doc2: 0.24494897427831785
Similitud coseno con doc1: 0.8951435925492911
Similitud coseno con doc2: 0.8846153846153845


## 3. Normalización de Vectores

Muchos sistemas normalizan los vectores para que su norma sea 1:

$\[ \hat{\vec{v}} = \frac{\vec{v}}{\|\vec{v}\|} \]$

Esto transforma la distancia Euclidiana en una función directa de la similitud coseno, facilitando búsquedas eficientes y comparables.

In [ ]:
def normalize(v):
    return v / norm(v)

q_norm = normalize(query)
d1_norm = normalize(doc1)
d2_norm = normalize(doc2)

print("Vector normalizado q:", q_norm)
print("Similitud coseno post-normalización (dot):", np.dot(q_norm, d1_norm), np.dot(q_norm, d2_norm))

# Relación teórica: dist² = 2 - 2cos(θ)
dot = np.dot(q_norm, d1_norm)
euclidean_sq = norm(q_norm - d1_norm)**2
print("2 - 2cos(theta):", 2 - 2 * dot)
print("Distancia euclidiana al cuadrado:", euclidean_sq)

Vector normalizado q: [0.19611614 0.58834841 0.78446454]
Similitud coseno post-normalización (dot): 0.8951435925492911 0.8846153846153845
2 - 2cos(theta): 0.2097128149014178
Distancia euclidiana al cuadrado: 0.2097128149014178


## 4. Indexación y Aceleración

Buscar en millones de vectores directamente es costoso $(\( O(n \cdot d) \))$. Se usan estructuras aproximadas para acelerar:

### a. IVF (Inverted File Index)
- Aplica clustering (K-means) a los vectores.
- Durante la búsqueda, se consulta solo un subconjunto de clústeres.

### b. HNSW (Hierarchical Navigable Small World)
- Construye un grafo jerárquico de vecinos más cercanos.
- Permite búsquedas logarítmicas eficientes.